<a href="https://colab.research.google.com/github/sellerstx1982/multimodal_damage_identification/blob/Vargas/project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imported Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from pathlib import Path
from sklearn.metrics import classification_report, balanced_accuracy_score, accuracy_score, classification_report
from PIL import Image, ImageFile
import os
import matplotlib.pyplot as plt
import zipfile
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, GlobalAveragePooling2D, Concatenate, Conv2D, MaxPooling2D, Flatten, Dropout, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import ResNet50, InceptionV3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam
import pickle
from flask import Flask, request, jsonify

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Load the data from pkl files
def load_data_from_pkl(pkl_filename):
    with open(pkl_filename, 'rb') as f:
        df = pickle.load(f)
    return df

In [4]:
df = load_data_from_pkl('/content/drive/MyDrive/Project_3/disaster_data.pkl')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 4436 to 4841
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   image    1000 non-null   object
 1   text     1000 non-null   object
 2   label    1000 non-null   object
 3   label_2  1000 non-null   object
 4   label_3  1000 non-null   object
 5   label_4  1000 non-null   object
dtypes: object(6)
memory usage: 54.7+ KB


In [6]:
df.head()

,image,text,label,label_2,label_3,label_4
4436,"[[[[0.21176471 0.15294118 0.14117647], [0.2 ...",#architecture #architectureporn #architecturel...,building,non-damage,non-damage,non-fire
5297,"[[[[0.12941176 0.20392157 0.36078431], [0.1294...",#thanksgiving #food #bomb #love #family #prebl...,food,non-damage,non-damage,non-fire
1121,"[[[[0.29019608 0.37254902 0.35294118], [0.2901...",Rusty car wreck somewhere between Cape Cross a...,wreckedcar,wreckedcar,damage,non-fire
561,"[[[[0.86666667 0.9372549 0.94509804], [0.8666...",Memories of #hurricanesandy,hurricanesandy,hurricane,damage,non-fire
1188,"[[[[0.98823529 1. 1. ], [0.9960...",Check the mess the wind left me #StormyScotlan...,naturaldisaster,naturaldisaster,damage,non-fire


Damage and Non-Damage Model Training

In [7]:
X_image = df['image']

resized_X_image = []

for img in X_image:
    if img.shape != (250, 250, 3):

        resized_img = np.resize(img, (250, 250, 3))

        resized_X_image.append(resized_img)
    else:

        resized_X_image.append(img)

X_images = np.array(resized_X_image)

In [8]:
# Convert the image column to a NumPy array
X_data = np.array([np.squeeze(image, axis=0) if image.shape[0] == 1 else image for image in X_images])

y_data = df['label_3'].values  # Convert labels to NumPy array

# Split the data into training and testing sets (75% training, 25% testing)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.25, random_state=42, stratify=y_data)

# Check the shapes of the training and test data
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (750, 250, 250, 3)
X_test shape: (250, 250, 250, 3)
y_train shape: (750,)
y_test shape: (250,)


In [9]:
# Convert categorical labels to numeric
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [10]:
# Define input shape
input_shape = (250, 250, 3)

# Define the input layer
inputs = Input(shape=input_shape)

# Load InceptionV3 model without the top layers and pretrained on ImageNet
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=inputs)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Flatten the output of the base model
x = Flatten()(base_model.output)

# Add fully connected layers with LeakyReLU and Dropout
x = Dense(64)(x)
x = LeakyReLU(alpha=0.1)(x)
x = Dropout(0.5)(x)
x = Dense(32)(x)
x = LeakyReLU(alpha=0.1)(x)

# Add the output layer for binary classification
outputs = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs, outputs)

# Compile the model with Adam optimizer and binary crossentropy loss
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Summarize the model
model.summary()

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 250, 250, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 124, 124, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 124, 124, 32)   │             96 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 124, 124, 32)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 122, 122, 32)   │          9,216 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 122, 122, 32)   │             96 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 122, 122, 32)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 122, 122, 64)   │         18,432 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 122, 122, 64)   │            192 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 122, 122, 64)   │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 60, 60, 64)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 60, 60, 80)     │          5,120 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 60, 60, 80)     │            240 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 60, 60, 80)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 58, 58, 192)    │        138,240 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 58, 58, 192)    │            576 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activat

 Total params: 26,523,553 (101.18 MB)

 Trainable params: 4,720,769 (18.01 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [11]:
history = model.fit(X_train,
                    y_train,
                    epochs=10,
                    batch_size=32)

Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 24s 765ms/step - accuracy: 0.6010 - loss: 0.8300
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 18s 765ms/step - accuracy: 0.8128 - loss: 0.4033
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 18s 762ms/step - accuracy: 0.8996 - loss: 0.2604
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 18s 763ms/step - accuracy: 0.9327 - loss: 0.2090
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 19s 791ms/step - accuracy: 0.9618 - loss: 0.1200
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 18s 767ms/step - accuracy: 0.9705 - loss: 0.1019
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 19s 772ms/step - accuracy: 0.9805 - loss: 0.0764
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 18s 758ms/step - accuracy: 0.9725 - loss: 0.0753
Epoch 9/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 19s 775ms/step - accuracy: 0.9867 - loss: 0.0449
Epoch 10/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 18s 764ms/step - accuracy: 0.9886 - loss: 0.0345


In [12]:
model.evaluate(X_test, y_test)

8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 756ms/step - accuracy: 0.8601 - loss: 0.6197


[0.5606962442398071, 0.8759999871253967]

LSTM Text Generator

In [ ]:
def create_lstm_model(image_feature_size, vocab_size, seq_length):
    image_input = layers.Input(shape=(image_feature_size,))
    text_input = layers.Input(shape=(seq_length,))

    # Combine Image and Text features
    x = layers.Concatenate()([image_input, text_input])
    x = layers.Embedding(vocab_size, 256)(x)
    x = layers.LSTM(256, return_sequences=True)(x)
    x = layers.Dense(vocab_size, activation='softmax')(x)

    model = models.Model(inputs=[image_input, text_input], outputs=x)
    return model

vocab_size = len(tokenizer)  # GPT tokenizer vocab size
seq_length = tokenized_reports['input_ids'].shape[1]  # Max sequence length
lstm_model = create_lstm_model(256, vocab_size, seq_length)

lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
gpt2_model = TFGPT2LMHeadModel.from_pretrained('gpt2')

# Fine-tune GPT-2 model
def fine_tune_gpt2(image_features, texts, tokenizer):
    inputs = tokenizer(texts, return_tensors="tf", padding=True, truncation=True)
    outputs = gpt2_model(inputs['input_ids'], labels=inputs['input_ids'])
    loss = outputs.loss
    logits = outputs.logits

    # Train your model on this loss
    return loss

# Example training loop
for epoch in range(10):
    loss = fine_tune_gpt2(image_features, sample_reports, tokenizer)
    print(f"Epoch {epoch+1}, Loss: {loss}")

In [ ]:
# Save the CNN + LSTM model or GPT-2 based model
lstm_model.save("disaster_report_model.h5")

# For GPT-2 (Hugging Face)
gpt2_model.save_pretrained("disaster_gpt2_model")
tokenizer.save_pretrained("disaster_gpt2_model_tokenizer")